In [346]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [347]:
from selenium import webdriver
import time

In [348]:
import pandas as pd
import numpy as np

In [349]:
from datetime import datetime, timezone, timedelta
import pytz

In [350]:
import uuid
import requests
import json

In [351]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [352]:
solr_base_url = "http://localhost:8983/solr/newsdb"
solr_update_url = solr_base_url + "/update?commit=true"
solr_select_url = solr_base_url + "/select"

In [353]:
def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    # "[Some] _ Article's Title--"
    # "[some] _ article's title--"
    s = s.lower()

    # "[some] _ article's_title--"
    # "[some]___article's_title__"
    for c in [' ', '-', '.', '/']:
        s = s.replace(c, '_')

    # "[some]___article's_title__"
    # "some___articles_title__"
    s = re.sub('\W', '', s)

    # "some___articles_title__"
    # "some   articles title  "
    s = s.replace('_', ' ')

    # "some   articles title  "
    # "some articles title "
    s = re.sub('\s+', ' ', s)

    # "some articles title "
    # "some articles title"
    s = s.strip()

    # "some articles title"
    # "some-articles-title"
    s = s.replace(' ', '-')

    return s

In [354]:
def deformat_timestr(timestr):
    return datetime.strptime(timestr, '%d %b %Y, %H:%M hrs IST')

In [355]:
def is_today(article_date, days):
    intz = pytz.timezone('Asia/Kolkata')
    now_dt = datetime.now(intz) - timedelta(days=days)
    article_dt = deformat_timestr(article_date)
    return now_dt.day - article_dt.day <= days

In [356]:
is_today("27 Dec 2018, 06:43 hrs IST", 2)

True

In [357]:
cities = ["delhi", "mumbai", "bangalore", "hyderabad", "kolkata", "chennai"]
toi_city_url = "https://timesofindia.indiatimes.com/city/{}"
toi_base_url = "https://timesofindia.indiatimes.com"
ltst_th = "https://www.thehindu.com/news/"
days = 30

In [358]:
def fetch_toi_daily_news(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    name_box = soup.find(attrs={'id': 'c_articlelist_stories_2'})
    # text_a = [s.get('href').strip() + '|' + s.getText().strip() for s in name_box.findAll('a', href=True)]
    lis = [s for s in name_box.findAll('li')]
    for li in lis:
        data = []
        found = False
        for s in li.findAll('a', href=True):
            article_link = toi_base_url + s.get('href').strip() 
            article_title = s.getText().strip()
            article_id = slugify(article_title)
            data = [article_id, article_title, article_link]
            found = True
            break
        if found:
            for s in li.findAll(attrs={'class': 'strlastupd'}):
                article_date = s.getText().strip()
                if not is_today(article_date, days):
                    return ""
                data.append(article_date)
                news_data.append(data)
    ul_page = name_box.find(attrs={'class': 'pagination'})
    lis = [s for s in ul_page.findAll('li')]
    pages = []
    next_link = None
    found_curr = False
    for li in lis:
        if found_curr:
            for s in li.findAll('a', href=True):
                next_link = toi_base_url + s.get('href').strip()
                break
            break
        curr = li.get('class')
        if curr:
            found_curr = True
    return next_link

In [359]:
def update_data_to_solr(news_data):
    count = 0
    batch_data = []
    for nd in news_data:
        doc_id, title, doc_url, updated_at, doc_city = nd[0], nd[1], nd[2], deformat_timestr(nd[3]).isoformat(), city
        solr_id = uuid.uuid4().hex
        body = {"id": solr_id, "doc_id": doc_id, "title": title, "doc_url": doc_url, "updated_at": updated_at, "city": doc_city}
        batch_data.append(body)
        count += 1
        if count%10 == 0:
            headers = {'Content-type': 'application/json'}
            r = requests.post(solr_update_url, data=json.dumps(batch_data), headers=headers)
            print(count, r.status_code)
            if r.status_code == 200:
                batch_data = []

In [360]:
def save_data_as_csv(news_data, city):
    # create a dataframe, save data to file
    news_df = pd.DataFrame(news_data)
    news_df.columns = ["article_id", "article_title", "article_url", "article_time"]

    file_name = "{}-initial.csv".format(city)
    news_df.to_csv(file_name, sep='|', encoding='utf-8', index=False)
    # df = pd.read_csv(file_name, delimiter='|')
    # df.head()

In [ ]:
for city in cities:
    # TODO: delhi has already been saved, remove this condition in future
    if city == "delhi":
        continue
    
    # scrape news data for the city
    news_data = []
    url = toi_city_url.format(city)
    continue_now = True
    count = 0
    while continue_now and count < 30:
        time.sleep(5)
        print(url)
        url = fetch_toi_daily_news(url)
        if news_data:
            print(news_data[-1])
        else:
            print("no data")
        if not url:
            continue_now = False
        count += 1
    
    # save the news data as csv file too
    save_data_as_csv(news_data, city)
    
    # add the news data to solr
    update_data_to_solr(news_data)

https://timesofindia.indiatimes.com/city/mumbai
['4-women-among-5-killed-as-fire-breaks-out-in-mumbai-high-rise', '4 women among 5 killed as fire breaks out in Mumbai high-rise', 'https://timesofindia.indiatimes.com/city/mumbai/4-women-among-5-killed-as-fire-breaks-out-in-mumbai-high-rise/articleshow/67279731.cms', '28 Dec 2018, 08:30 hrs IST']
https://timesofindia.indiatimes.com/city/mumbai/2
['bodies-of-missing-it-firm-staffers-found-at-khadavali-river-in-kalyan', 'Bodies of missing IT firm staffers found at Khadavali river in Kalyan', 'https://timesofindia.indiatimes.com/city/mumbai/bodies-of-missing-it-firm-staffers-found-at-khadavali-river-in-kalyan/articleshow/67238289.cms', '25 Dec 2018, 04:48 hrs IST']
https://timesofindia.indiatimes.com/city/mumbai/3
['1800-students-taking-law-exam-given-question-paper-1-hour-late-in-mumbai', '1,800 students taking law exam given question paper 1 hour late in Mumbai', 'https://timesofindia.indiatimes.com/city/mumbai/1800-students-taking-law-ex

['when-chitrangda-singh-rocked-the-stage-at-the-bangalore-times-fashion-week', 'When Chitrangda Singh rocked the stage at the Bangalore Times Fashion Week', 'https://timesofindia.indiatimes.com/city/bengaluru/when-chitrangda-singh-rocked-the-stage-at-the-bangalore-times-fashion-week/articleshow/66920121.cms', '4 Dec 2018, 00:00 hrs IST']
10 200
20 200
30 200
40 200
50 200
60 200
70 200
80 200
90 200
100 200
110 200
120 200
130 200
140 200
150 200
160 200
170 200
180 200
190 200
200 200
210 200
220 200
230 200
240 200
250 200
260 200
270 200
280 200
290 200
300 200
310 200
320 200
330 200
340 200
350 200
360 200
370 200
380 200
390 200
400 200
410 200
420 200
430 200
440 200
450 200
460 200
470 200
480 200
490 200
500 200
510 200
520 200
530 200
540 200
550 200
https://timesofindia.indiatimes.com/city/hyderabad
['dalmatian-left-in-locked-house-for-7-days-condition-critical-says-vet', 'Dalmatian left in locked house for 7 days; condition critical, says vet', 'https://timesofindia.indiati

553

In [321]:
titles_df = news_df.loc[:, ["article_id" ,"article_title"]]

In [322]:
titles_df.head()

,article_id,article_title
0,gangster-act-in-cow-slaughter-cases-in-ghaziabad,Gangster Act in cow slaughter cases in Ghaziabad
1,delhi-governments-water-body-revival-project-b...,Delhi government’s water body revival project ...
2,department-for-pwd-its-work-in-progress,Department for PwD: It's work in progress
3,34-year-old-shot-outside-his-house,34-year-old shot outside his house
4,transgenders-can-file-sexual-harassment-case-c...,Transgenders can file sexual harassment case: ...


In [323]:
lemmatizer = WordNetLemmatizer()
stopset = set(stopwords.words('english'))

In [324]:
def process(txt):
    tokens = []
    for token in wordpunct_tokenize(txt):
        if token.isdigit():
            continue
        if all(char in string.punctuation for char in token):
            continue
        if len(token) == 1:
            continue
        token = token.lower()
        token = token.strip()  # Strip whitespace and other punctuations
        token = token.strip('_')  # remove _ if any
        token = token.strip('*')
        if token in stopset:
            continue
        tokens.append(token)
        lemmatizer.lemmatize(token)
    return tokens

In [325]:
titles_df["atoken_title"] = titles_df["article_title"].apply(process)
titles_df['aprocessed_title'] = titles_df['atoken_title'].apply(lambda tokens: ' '.join(str(v) for v in tokens))

In [326]:
titles_df.head()

,article_id,article_title,atoken_title,aprocessed_title
0,gangster-act-in-cow-slaughter-cases-in-ghaziabad,Gangster Act in cow slaughter cases in Ghaziabad,"[gangster, act, cow, slaughter, cases, ghaziabad]",gangster act cow slaughter cases ghaziabad
1,delhi-governments-water-body-revival-project-b...,Delhi government’s water body revival project ...,"[delhi, government, water, body, revival, proj...",delhi government water body revival project br...
2,department-for-pwd-its-work-in-progress,Department for PwD: It's work in progress,"[department, pwd, work, progress]",department pwd work progress
3,34-year-old-shot-outside-his-house,34-year-old shot outside his house,"[year, old, shot, outside, house]",year old shot outside house
4,transgenders-can-file-sexual-harassment-case-c...,Transgenders can file sexual harassment case: ...,"[transgenders, file, sexual, harassment, case,...",transgenders file sexual harassment case cops hc


In [327]:
aprocess_titles_df = titles_df.loc[:, ["aprocessed_title"]]
train_df = aprocess_titles_df

In [328]:
no_features = 50
tfidf_vectorizer = CountVectorizer(max_features=no_features)

def vectorize(txt, vectorizer):
    X = vectorizer.fit_transform(txt)
    feature_names = vectorizer.get_feature_names()
    return X, feature_names

In [329]:
X_tf, tf_feature_names = vectorize(train_df['aprocessed_title'], tfidf_vectorizer)

In [330]:
no_topics = 10
num_iter = 10

In [331]:
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=num_iter, learning_method='online', learning_offset=50.,random_state=9, evaluate_every=100).fit(X_tf)

In [332]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_features = [(feature_names[i], topic[i]) for i in topic.argsort()[:-no_top_words - 1:-1]]
        print("Topic %d:" % (topic_idx))
        print(", ".join([str(val[0])+": "+"%.2f" % val[1] for val in top_features]))

In [334]:
display_topics(lda, tf_feature_names, 10)

Topic 0:
hc: 13.83, government: 7.11, held: 5.68, asks: 5.34, case: 1.64, cops: 1.39, may: 0.93, govt: 0.23, murder: 0.21, delhi: 0.21
Topic 1:
new: 12.50, says: 6.17, death: 5.56, fire: 1.69, govt: 0.99, driver: 0.90, asks: 0.25, fog: 0.21, delhi: 0.20, parking: 0.20
Topic 2:
delhi: 15.68, police: 14.18, three: 10.50, cm: 9.09, kejriwal: 7.40, panel: 7.37, near: 1.71, man: 1.03, may: 0.91, day: 0.62
Topic 3:
rs: 30.78, delhi: 25.24, crore: 14.01, lakh: 11.22, day: 10.37, christmas: 6.98, held: 6.36, woman: 3.38, new: 1.31, case: 0.34
Topic 4:
get: 8.82, metro: 0.91, near: 0.91, quality: 0.22, air: 0.20, delhi: 0.19, school: 0.19, cop: 0.18, years: 0.18, driver: 0.18
Topic 5:
aap: 21.07, bjp: 10.01, fog: 5.78, kejriwal: 2.44, car: 1.74, government: 1.12, govt: 0.92, asks: 0.41, metro: 0.40, case: 0.28
Topic 6:
sajjan: 7.99, murder: 7.88, parking: 7.62, kumar: 7.17, case: 6.73, hc: 3.94, man: 2.49, death: 1.58, three: 0.48, delhi: 0.28
Topic 7:
delhi: 28.62, school: 11.58, south: 8.52, 